In [1]:
# Retraining NN with Elegy then do HMCon Mana
# Author: Peter Nov 5 2022

# New Requirements:
# conda install python=3.9 numpy scipy pandas matplotlib
# conda install -c anaconda cudatoolkit
# pip install tensorflow
# pip install tensorflow-io\[tensorflow\] # Seems to want specific older tf versions

# pip install elegy==0.8.5 # Because 0.8.6 has error.
# pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
# pip install tfp-nightly tensorflow_io tensorflow

"""
Train NN.
Peter July 2023

New Requirements:
conda install python=3.9 numpy scipy pandas matplotlib
conda install -c anaconda cudatoolkit
pip install tensorflow
pip install tensorflow-io\[tensorflow\] # Seems to want specific older tf versions


"""

import os
from collections import defaultdict
import numpy as np
import h5py
import matplotlib.pyplot as plt

import keras_core as keras


import tensorflow_io as tfio
from tensorflow.data import Dataset
from tensorflow.data.experimental import AUTOTUNE

#import tensorflow as tf 
# #os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
# #os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.10'
# #os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# import jax
# import jax.numpy as jnp
# from jax import random
# from jax import vmap
# from jax import jit
# from jax import grad
# #assert jax.default_backend() == 'gpu'

# import elegy # pip install elegy
# import optax
# import tensorflow_io as tfio # pip install tensorflow-io
# #import tensorflow as tf # Recommended not to import this with jax because will also try to grab memory.
# from tensorflow.data import Dataset # Trying not to import tf. 
# 

2023-07-13 00:04:45.428405: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-13 00:04:45.432699: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-13 00:04:45.481617: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-13 00:04:45.484854: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 00:04:46.418526: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Using TensorFlow backend


In [2]:

# Create dataset object using IODataset
polarity = 'neg'
path = '/mnt/lts/nfs_fs02/sadow_lab/shared/gcr/data/2023_07_01/'
f = f'{path}/{polarity}/model_collection_1AU_90deg_0deg_fixed_training.h5'
# 8 input parameters for the NN: alpha, cmf, vspoles, cpa, pwr1par, pwr2par, pwr1perr, and pwr2perr.
#features = ['alpha', 'cmf', 'cpa', 'pwr1par', 'pwr1perr', 'pwr2par', 'pwr2perr', 'vspoles']
with h5py.File(f, 'r') as h5:
    num_samples, num_inputs,  = h5['X_minmax'].shape
    _, num_flux,  = h5['Y_log_scaled'].shape
x = tfio.IODataset.from_hdf5(f, dataset='/X_minmax')
y = tfio.IODataset.from_hdf5(f, dataset='/Y_log_scaled')

# Split
full = Dataset.zip((x, y))
train = full.take(np.floor(num_samples *.9))#.repeat()
test = full.skip(np.floor(num_samples *.9))#.repeat()

# Batch
BATCH_SIZE = 128
train = train.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)
test = test.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)

# Some calcs
steps_per_epoch = int(num_samples * .9 / BATCH_SIZE )
validation_steps = int(num_samples * .1 / BATCH_SIZE)
print(f'Steps per epoch: {steps_per_epoch}')

#train_x = TFDatasetAdapter(train.map(lambda x,y: x))
#train_y = TFDatasetAdapter(train.map(lambda x,y: x))
#x = train.map(lambda x,y: x)
#y = train.map(lambda x,y: y)

2023-07-13 00:04:48.687014: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2023-07-13 00:04:48.687147: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 AVX512F FMA
2023-07-13 00:04:48.924886: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Steps per epoch: 13975


In [3]:
# Define model. 
l2=keras.regularizers.L2(l2=1e-6)
model = keras.Sequential(layers=[
   keras.layers.Input(shape=(8,)),
   keras.layers.Dense(256, activation='selu', kernel_regularizer=l2),
   keras.layers.Dense(256, activation='selu', kernel_regularizer=l2),
   keras.layers.Dense(32, activation='linear', kernel_regularizer=l2),
])

optimizer = keras.optimizers.Adam(learning_rate=1e-4)
model_path = f'../models/model_v1.0_{polarity}.keras'  # Must end with keras.
callbacks = [keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10),
             keras.callbacks.EarlyStopping(monitor="val_loss", patience=20),
             keras.callbacks.ModelCheckpoint(filepath=model_path, save_best_only=True, monitor='val_loss'),
            ]
model.compile(loss='mse', optimizer=optimizer)

history = model.fit(
    train,
    epochs=100,
    #steps_per_epoch=steps_per_epoch, #6 * 10000, #10000, # 10k*128 is approximate size of training set.
    validation_data=test,
    #validation_steps=1000,
    shuffle=False,
    verbose=2,
    callbacks=callbacks,
)


Epoch 1/100


2023-07-13 00:04:49.229330: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_32' with dtype int64 and shape [2]
	 [[{{node Placeholder/_32}}]]
2023-07-13 00:04:49.229993: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [1]
	 [[{{node Placeholder/_10}}]]


2023-07-13 00:04:49.766160: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x2b77c4028510 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-07-13 00:04:49.766183: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Host, Default Version
2023-07-13 00:04:49.842116: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-13 00:04:50.188693: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
/home/psadow/anaconda3/envs/gcr/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(t

13975/13975 - 33s - 2ms/step - loss: 0.0014 - val_loss: 3.0122e-04 - learning_rate: 1.0000e-04
Epoch 2/100
13975/13975 - 33s - 2ms/step - loss: 2.5170e-04 - val_loss: 2.0559e-04 - learning_rate: 1.0000e-04
Epoch 3/100
13975/13975 - 20s - 1ms/step - loss: 1.7996e-04 - val_loss: 1.5387e-04 - learning_rate: 1.0000e-04
Epoch 4/100
13975/13975 - 20s - 1ms/step - loss: 1.3656e-04 - val_loss: 1.1912e-04 - learning_rate: 1.0000e-04
Epoch 5/100
13975/13975 - 19s - 1ms/step - loss: 1.0742e-04 - val_loss: 9.5646e-05 - learning_rate: 1.0000e-04
Epoch 6/100
13975/13975 - 20s - 1ms/step - loss: 8.7409e-05 - val_loss: 7.9320e-05 - learning_rate: 1.0000e-04
Epoch 7/100
13975/13975 - 22s - 2ms/step - loss: 7.3320e-05 - val_loss: 6.7807e-05 - learning_rate: 1.0000e-04
Epoch 8/100
13975/13975 - 26s - 2ms/step - loss: 6.3235e-05 - val_loss: 5.9145e-05 - learning_rate: 1.0000e-04
Epoch 9/100
13975/13975 - 20s - 1ms/step - loss: 5.5882e-05 - val_loss: 5.2829e-05 - learning_rate: 1.0000e-04
Epoch 10/100
1397

In [33]:
# Test model load.
model_path = '../models/model_2_256_selu_l21e-6.keras'  # Must end with keras.
model2 = keras.models.load_model(model_path)
x = np.random.rand(1,8)
yhat = model.predict(x)
print(yhat, yhat.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
[[ 0.37365347  0.37716502  0.38417703  0.39466333  0.41205156  0.43274304
   0.45662355  0.4868097   0.51950806  0.5541598   0.5928316   0.6309504
   0.66670454  0.69956523  0.72432494  0.738734    0.7411985   0.7304528
   0.707007    0.66976994  0.6221982   0.55842876  0.48455873  0.39619845
   0.29504338  0.18976489  0.07533902 -0.04410985 -0.17151965 -0.30297035
  -0.43793932 -0.57118696]] (1, 32)


In [ ]:
# Make predictions.

path = '/mnt/lts/nfs_fs02/sadow_lab/shared/gcr/data/2023_07_01/'
infile = f'{path}/pos/model_collection_1AU_90deg_0deg_fixed.h5'
outfile = f'{path}/pos/model_collection_1AU_90deg_0deg_fixed_processed.h5'